In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import MCM

In [5]:
dict = MCM.load('D:/Files/code/Python/MCM/file/weight-save-2024-02-03-09-02-18.json')
origin_data = pd.read_csv('D:/Files/code/Python/MCM/work/Wimbledon_featured_matches.csv')
# 'p1_m_avg':origin_data['p1_m_avg'],'p2_m_avg':origin_data['p2_m_avg'],'p1_m_gaussian':origin_data['p1_m_gaussian'],'p2_m_gaussian':origin_data['p2_m_gaussian']
df = pd.DataFrame()
df['p1_m_avg'] = dict['p1_m_avg']
df['p2_m_avg'] = dict['p2_m_avg']
df['p1_m_gaussian'] = dict['p1_m_gaussian']
df['p2_m_gaussian'] = dict['p2_m_gaussian']
df['same'] = 0
for i in range(1,len(origin_data)):
    if origin_data['point_victor'][i] == origin_data['point_victor'][i-1]:
        df.loc[i,'same'] = 1
df

,p1_m_avg,p2_m_avg,p1_m_gaussian,p2_m_gaussian,same
0,0.161460,1.692221,0.563792,1.339982,0
1,1.254219,0.693088,0.958618,1.039164,0
2,0.382009,1.690079,0.833377,1.272500,0
3,1.244721,0.705944,1.566132,0.777732,0
4,2.365282,0.693088,1.636516,1.045027,1
...,...,...,...,...,...
7279,1.325717,0.669628,1.267689,0.739879,0
7280,1.330769,0.663512,1.061099,0.989983,1
7281,0.550596,1.608015,0.884178,1.204170,0
7282,1.334137,0.659435,1.333501,0.660204,0


In [6]:
next_days = 20
break_p1_m_avg = []
no_break_p1_m_avg = []
break_p2_m_avg = []
no_break_p2_m_avg = []
game_begin = 0
p1_last = -1
p2_last = -1
for i in range(len(origin_data)):
    this = False
    if origin_data['p1_break_pt_won'][i] == 1:
        if p1_last == -1:
            p1_last = i
            continue
        if origin_data['match_id'][i] != origin_data['match_id'][p1_last]:
            p1_last = i
            continue
        between = int((i - p1_last - 1) / 2)
        for j in range(p1_last+1,p1_last+1+between):
            break_p1_m_avg.append(df['p1_m_avg'][j])
        for j in range(p1_last+1+between,p1_last+1+2*between):
            no_break_p1_m_avg.append(df['p1_m_avg'][j])
        p1_last = i
    if origin_data['p2_break_pt_won'][i] == 1:
        if p2_last == -1:
            p2_last = i
            continue
        if origin_data['match_id'][i] != origin_data['match_id'][p2_last]:
            p2_last = i
            continue
        between = int((i - p2_last - 1) / 2)
        for j in range(p2_last+1,p2_last+1+between):
            break_p2_m_avg.append(df['p2_m_avg'][j])
        for j in range(p2_last+1+between,p2_last+1+2*between):
            no_break_p2_m_avg.append(df['p2_m_avg'][j])
        p2_last = i
        
MCM.print("breakp1和nobreakp1数据个数:",break_p1_m_avg.__len__())
MCM.print("breakp2和nobreakp2数据个数:",break_p2_m_avg.__len__())
from scipy.stats import kstest, ks_2samp, mannwhitneyu, median_test, anderson_ksamp
import scipy as sp
MCM.print("数据划分说明:当一场比赛内有多个来自同一人的break point时,将break point间的数据均分为两组,离break point近的一组为break point组,远的一组为no break point组")
MCM.print("kstest检验内容:是否符合正态分布,按照breakp1,nobreakp1,breakp2,nobreakp2顺序") #非常不符合正态分布
MCM.print(kstest(break_p1_m_avg,'norm'),kstest(no_break_p1_m_avg,'norm'))
MCM.print(kstest(break_p2_m_avg,'norm'),kstest(no_break_p2_m_avg,'norm'))
MCM.print("以下检验均按照breakp1-nobreakp1,breakp2-nobreakp2顺序")
MCM.print("ks_2samp检验内容:两组数据的分布是否有显著差异") #具有显著差异
MCM.print(ks_2samp(break_p1_m_avg,no_break_p1_m_avg))
MCM.print(ks_2samp(break_p2_m_avg,no_break_p2_m_avg))
MCM.print("mannwhitneyu检验内容:两组数据的形状是否有显著差异") #具有显著差异
MCM.print(mannwhitneyu(break_p1_m_avg,no_break_p1_m_avg))
MCM.print(mannwhitneyu(break_p2_m_avg,no_break_p2_m_avg))
MCM.print("median_test检验内容:两组数据的中位数是否有显著差异") #具有显著差异
MCM.print(median_test(break_p1_m_avg,no_break_p1_m_avg))
MCM.print(median_test(break_p2_m_avg,no_break_p2_m_avg))
MCM.print("anderson_ksamp检验内容:两组数据的分布是否有显著差异") #具有显著差异
MCM.print(anderson_ksamp([break_p1_m_avg,no_break_p1_m_avg],method=sp.stats.PermutationMethod()))
MCM.print(anderson_ksamp([break_p2_m_avg,no_break_p2_m_avg],method=sp.stats.PermutationMethod()))


breakp1和nobreakp1数据个数: 1581
breakp2和nobreakp2数据个数: 1122
数据划分说明:当一场比赛内有多个来自同一人的break point时,将break point间的数据均分为两组,离break point近的一组为break point组,远的一组为no break point组
kstest检验内容:是否符合正态分布,按照breakp1,nobreakp1,breakp2,nobreakp2顺序
KstestResult(statistic=0.5553216913936474, pvalue=0.0, statistic_location=0.1423208567142409, statistic_sign=-1) KstestResult(statistic=0.5858040070443035, pvalue=0.0, statistic_location=0.45197237099322335, statistic_sign=-1)
KstestResult(statistic=0.5547724766816231, pvalue=0.0, statistic_location=0.14901424742674682, statistic_sign=-1) KstestResult(statistic=0.5689254355166478, pvalue=0.0, statistic_location=0.3258972846997668, statistic_sign=-1)
以下检验均按照breakp1-nobreakp1,breakp2-nobreakp2顺序
ks_2samp检验内容:两组数据的分布是否有显著差异
KstestResult(statistic=0.11385199240986717, pvalue=2.423158657032965e-09, statistic_location=1.3973805008395408, statistic_sign=1)
KstestResult(statistic=0.08467023172905526, pvalue=0.0006383862498878363, statistic_location=1.4633420680354958, stati

In [7]:
next_days = 20
break_p1_m_avg = []
no_break_p1_m_avg = []
break_p2_m_avg = []
no_break_p2_m_avg = []
game_begin = 0
finish = -1
i = 0
origin_data_len = len(origin_data)
while(i < origin_data_len-2):
    if origin_data['match_id'][i] != origin_data['match_id'][i+1]:
        i+=1
        continue
    break_p1_m_avg.append(df['p1_m_avg'][i])
    break_p2_m_avg.append(df['p2_m_avg'][i])
    i+=1
    no_break_p1_m_avg.append(df['p1_m_avg'][i])
    no_break_p2_m_avg.append(df['p2_m_avg'][i])
    i+=1
        
MCM.print("group1p1和group2p2数据个数",break_p1_m_avg.__len__())
MCM.print("group1p2和group2p2数据个数",break_p2_m_avg.__len__())
from scipy.stats import kstest, ks_2samp, mannwhitneyu, median_test, anderson_ksamp
import scipy as sp
MCM.print("数据划分说明:将一场比赛内的所有数据，交替划分为两组，如果一场比赛内的数据为奇数，则最后一个数据不参与划分")
MCM.print("kstest检验内容:是否符合正态分布,按照group1p1,group2p1,group1p2,group2p2顺序") #非常不符合正态分布
MCM.print(kstest(break_p1_m_avg,'norm'),kstest(no_break_p1_m_avg,'norm'))
MCM.print(kstest(break_p2_m_avg,'norm'),kstest(no_break_p2_m_avg,'norm'))
MCM.print("以下检验均按照group1p1-group2p1,group1p2-group2p2顺序")
MCM.print("ks_2samp检验内容:两组数据的分布是否有显著差异") #具有显著差异
MCM.print(ks_2samp(break_p1_m_avg,no_break_p1_m_avg))
MCM.print(ks_2samp(break_p2_m_avg,no_break_p2_m_avg))
MCM.print("mannwhitneyu检验内容:两组数据的形状是否有显著差异") #具有显著差异
MCM.print(mannwhitneyu(break_p1_m_avg,no_break_p1_m_avg))
MCM.print(mannwhitneyu(break_p2_m_avg,no_break_p2_m_avg))
MCM.print("median_test检验内容:两组数据的中位数是否有显著差异") #具有显著差异
MCM.print(median_test(break_p1_m_avg,no_break_p1_m_avg))
MCM.print(median_test(break_p2_m_avg,no_break_p2_m_avg))
MCM.print("anderson_ksamp检验内容:两组数据的分布是否有显著差异") #具有显著差异
MCM.print(anderson_ksamp([break_p1_m_avg,no_break_p1_m_avg],method=sp.stats.PermutationMethod()))
MCM.print(anderson_ksamp([break_p2_m_avg,no_break_p2_m_avg],method=sp.stats.PermutationMethod()))

group1p1和group2p2数据个数 3633
group1p2和group2p2数据个数 3633
数据划分说明:将一场比赛内的所有数据，交替划分为两组，如果一场比赛内的数据为奇数，则最后一个数据不参与划分
kstest检验内容:是否符合正态分布,按照group1p1,group2p1,group1p2,group2p2顺序
KstestResult(statistic=0.5608334737680913, pvalue=0.0, statistic_location=0.29540999485026853, statistic_sign=-1) KstestResult(statistic=0.5609499809444333, pvalue=0.0, statistic_location=0.45349933706328754, statistic_sign=-1)
KstestResult(statistic=0.5553580778585397, pvalue=0.0, statistic_location=0.15315802296412362, statistic_sign=-1) KstestResult(statistic=0.5570318344097788, pvalue=0.0, statistic_location=0.288345944962842, statistic_sign=-1)
以下检验均按照group1p1-group2p1,group1p2-group2p2顺序
ks_2samp检验内容:两组数据的分布是否有显著差异
KstestResult(statistic=0.01486374896779521, pvalue=0.8171449632163493, statistic_location=0.6970855163346886, statistic_sign=1)
KstestResult(statistic=0.01844205890448665, pvalue=0.5671302382980045, statistic_location=1.397140604588117, statistic_sign=-1)
mannwhitneyu检验内容:两组数据的形状是否有显著差异
MannwhitneyuResul